In [1]:
import acquire
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
titanic = acquire.get_titanic_data()

In [3]:
titanic.head(3)

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1


### What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [6]:
titanic.survived.mode()[0]

0

In [9]:
titanic['model_baseline'] = titanic.survived.mode()[0]

In [10]:
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,model_baseline
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1,0
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1,0


### Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
train.head()

In [ ]:
X_train = validate.drop(columns=['survived'])

In [ ]:
y_train= train.survived

In [ ]:
clf = DecisionTreeClassifier(max_depth= 3, random_state=123)

In [ ]:
y_pred = clf.y_pred()

### Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
pd.concat([train.iloc[:,1:3], train.iloc[:,5:]], axis=1)

### Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
classification_report(model)

In [ ]:
TN, FP, FN, TP = conf.ravel()

### Run through steps 2-4 using a different max_depth value.

### Which model performs better on your in-sample data?

### Which model performs best on your out-of-sample data, the validate set?

### Work through these same exercises using the Telco dataset.

### Experiment with this model on other datasets with a higher number of output classes